In [164]:
# Author: Chun-Yi Chen
# Matriculation No.1820370

In [1]:
import pandas as pd
import requests
import bs4
import os
import re
import time
import random
import numpy as np

C:\Users\Austin\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
os.chdir("D:\\user\\Documents\\WebMining\\data")

In [3]:
file_list = []
for i in os.listdir():
    if "k.csv"in i:
        file_list.append(i)
print(file_list)

['book1-100k.csv', 'book1000k-1100k.csv', 'book100k-200k.csv', 'book1100k-1200k.csv', 'book1200k-1300k.csv', 'book1300k-1400k.csv', 'book1400k-1500k.csv', 'book1500k-1600k.csv', 'book1600k-1700k.csv', 'book1700k-1800k.csv', 'book1800k-1900k.csv', 'book1900k-2000k.csv', 'book2000k-3000k.csv', 'book200k-300k.csv', 'book3000k-4000k.csv', 'book300k-400k.csv', 'book4000k-5000k.csv', 'book400k-500k.csv', 'book500k-600k.csv', 'book600k-700k.csv', 'book700k-800k.csv', 'book800k-900k.csv', 'book900k-1000k.csv']


In [4]:
# get original columns for append
init_column = pd.read_csv(file_list[0]).columns
# create a big df for crawler and id-join
final_df = pd.DataFrame(columns=init_column)
for i in file_list:
    final_df = final_df.append(pd.read_csv(i))

In [5]:
final_df = final_df.reset_index(drop= True)
final_df = final_df.drop_duplicates()
final_df

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher,CountsOfReview,...,Language,Authors,Rating,RatingDist2,RatingDist5,ISBN,RatingDist3,Description,Count of text reviews,PagesNumber
0,1,Harry Potter and the Half-Blood Prince (Harry ...,1:9896,652,4:556485,total:2298124,16,9,Scholastic Inc.,28062,...,eng,J.K. Rowling,4.57,2:25317,5:1546466,NaN,3:159960,NaN,NaN,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,...,eng,J.K. Rowling,4.50,2:37005,5:1493113,0439358078,3:211781,NaN,NaN,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,1:108202,309,4:1513191,total:6587388,1,11,Scholastic Inc,75911,...,eng,J.K. Rowling,4.47,2:130310,5:4268227,NaN,3:567458,NaN,NaN,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,...,eng,J.K. Rowling,4.42,2:49353,5:1504505,0439554896,3:288821,NaN,NaN,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,...,eng,J.K. Rowling,4.57,2:24849,5:1749958,043965548X,3:194848,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850305,999991,Gendered Talk at Work: Constructing Gender Ide...,1:0,251,4:3,total:25,21,7,Wiley-Blackwell,0,...,NaN,Janet Holmes,4.28,2:1,5:15,1405117591,3:6,<i>Gendered Talk at Work</i> examines how wome...,0.0,NaN
1850306,999992,Good Will Toward Men: Women Talk Candidly abou...,1:0,231,4:0,total:0,1,2,St. Martin's Press,0,...,NaN,Jack Kammer,0.00,2:0,5:0,0312104715,3:0,In interviews with gender issues journalist Ja...,0.0,NaN
1850307,999993,Would You Rather . . . ?: The Outrageous Book ...,1:2,400,4:35,total:121,1,10,Workman Publishing Company,14,...,NaN,Randy Horn,3.81,2:11,5:39,0761124209,3:34,<i>Would You Rather...?</i> takes the idea of ...,14.0,NaN
1850308,999997,Addictions Treatment for Older Adults: Evaluat...,1:0,258,4:0,total:0,2,2,Routledge,0,...,NaN,Kathryn Graham,0.00,2:0,5:0,1560248572,3:0,Here is a detailed description of an innovativ...,0.0,NaN


In [6]:
Books = pd.read_csv("Books.csv")

C:\Users\Austin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
Genre_df = final_df.copy()
# drop NA value in ISBN
Genre_df= Genre_df.loc[Genre_df.ISBN.dropna().index,:]
# filter by ISBN
Genre_df = Genre_df.merge(Books,on = "ISBN")
# generate empty columns
Genre_df.loc[:,"Genre"] = ""
Genre_df.loc[:,"Genre_user_amount"] = ""

In [8]:
with open('proxy_list.txt', 'r') as file:
    proxy_ips = file.read().splitlines()
    print(proxy_ips)

['63.161.104.189:3128', '103.73.194.2:80', '1.179.232.220:8080', '194.233.69.90:443']


In [9]:
valid_index = []
for k,ip in enumerate(proxy_ips):
    try:
        result = requests.get('https://ip.seeip.org/jsonip?',
                              proxies={'http': ip, 'https': ip},
                              timeout=1.5)
        print(result.json())
        valid_index.append(k)
    except:

        print(f"{ip} invalid")

{'ip': '63.161.104.189'}
103.73.194.2:80 invalid
1.179.232.220:8080 invalid
194.233.69.90:443 invalid


In [10]:
proxy_ips = np.array(proxy_ips)

In [189]:
# get proxy
response = requests.get("https://www.sslproxies.org/")
 
proxy_ips = re.findall('\d+\.\d+\.\d+\.\d+:\d+', response.text)  
valid_ips = []
for ip in proxy_ips:

    try:
        result = requests.get('https://ip.seeip.org/jsonip?',
                              proxies={'http': ip, 'https': ip},
                              timeout=2.5)
        print(result.json())
        valid_ips.append(ip)
    except:
        print(f"{ip} invalid")

with open('proxy_list.txt', 'w') as file:
    for ip in valid_ips:
        file.write(ip + '\n')
    file.close()
    

2.187.219.116:8080 invalid
151.106.18.122:1080 invalid
151.106.17.124:1080 invalid
188.138.11.48:5566 invalid
151.106.18.126:1080 invalid
151.106.18.123:1080 invalid
{'ip': '103.73.194.2'}
{'ip': '128.199.214.87'}
160.16.242.164:3128 invalid
192.155.107.213:1080 invalid
151.106.13.221:1080 invalid
188.138.106.158:5566 invalid
146.120.174.149:8989 invalid
85.25.132.9:5566 invalid
{'ip': '185.220.101.62'}
151.106.17.123:1080 invalid
178.167.87.200:55443 invalid
188.138.90.226:5566 invalid
151.106.17.122:1080 invalid
216.176.187.99:8886 invalid
20.47.108.204:8888 invalid
192.155.107.212:1080 invalid
151.106.18.125:1080 invalid
{'ip': '170.155.100.128'}
151.106.13.220:1080 invalid
85.25.119.98:5566 invalid
85.25.4.27:5566 invalid
23.224.198.13:59394 invalid
151.106.17.125:1080 invalid
200.25.254.193:54240 invalid
85.25.226.133:5566 invalid
103.255.178.20:9080 invalid
85.25.132.27:5566 invalid
77.238.129.14:55443 invalid
200.54.194.10:53281 invalid
{'ip': '185.220.102.250'}
103.178.43.6:818

In [158]:
valid_ips = np.array(valid_ips)
print(valid_ips)

['45.80.149.36:23456' '20.116.130.70:3128' '116.203.201.82:8443'
 '130.61.95.193:3128' '178.167.87.200:55443' '213.32.75.44:9300']


In [14]:
valid_index = []
for k,ip in enumerate(valid_ips):
    try:
        result = requests.get('https://ip.seeip.org/jsonip?',
                              proxies={'http': ip, 'https': ip},
                              timeout=2.5)
        print(result.json())
        valid_index.append(k)
    except:

        print(f"{ip} invalid")

103.53.79.113:8080 invalid
138.0.188.134:10001 invalid
170.155.5.235:8080 invalid
{'ip': '59.124.224.205'}
{'ip': '63.161.104.189'}
8.213.128.19:80 invalid


In [15]:
# with open('proxy_list.txt', 'w') as file:
#     for ip in np.array(valid_ips)[valid_index]:
#         file.write(ip + '\n')
#     file.close()
# proxy_ips = np.array(valid_ips)[valid_index]
# valid_ips = np.array(valid_ips)

In [190]:
proxy_ips = valid_ips.copy()

In [245]:
# Crawler

def Crawler(ID, proxy_list):
    
    genre_string = ""
    genre_users_string = ""
    response = 0
    for i in range(10):
        cur_proxy = proxy_list[random.randint(0,len(proxy_list)-1)]
        try:
            response  = requests.get(f"https://www.goodreads.com/book/show/{ID}"
                                 ,proxies={'http': f'{cur_proxy}', 'https': f'{cur_proxy}'}
                                 ,timeout=3)
            break
        except:
            continue
    try:
        if response.status_code != 200:
            response  = requests.get(f"https://www.goodreads.com/book/show/{ID}",
                                    timeout=1.5)
    except:
        return "Fail connection","Fail connection"
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    genre_list = soup.find_all("a", {"class": "bookPageGenreLink"})
    genre_amount = soup.find_all("div", {"class": "bookPageGenreLink"})
    for i in range(min(len(genre_list),len(genre_amount))):
        genre_string += genre_list[i].text
        m = re.search(r"(\d{0,3},?\d{0,3},?\d{0,3}) users?",genre_amount[i].text)
        

        genre_users_string +=  m.group(1)

        genre_string += "|"
        genre_users_string += "|"
    return genre_string, genre_users_string


In [18]:
# import threading
# import requests
# import time
# import queue
# import concurrent

# dic_genre = {}
# dic_genre_number = {}
# start = time.time()
# # define function for thread
# class myThread(threading.Thread):
#     def __init__(self, name, q):
#         threading.Thread.__init__(self)
#         self.name = name
#         self.q = q
#     def run(self):
#         print("Starting " + self.name)
#         while True:
#             try:
#                 crawl(self.name, self.q)
#             except:
#                 break
#         print("Exiting " + self.name)

# def crawl(url):
#     global dic_genre, dic_genre_number

#     ID = url.split("/")[-1]
    
# #     try:
# #         r = requests.get(url, timeout=1.5)
# #         print(threadName, r.status_code, url)
# #     except Exception as e:
# #         print(threadName, "Error: ", e)
        
#     cur_proxy = proxy_ips[random.randint(0,len(proxy_ips)-1)]
#     genre_string = ""
#     genre_users_string = ""
#     for i in range(10):
#         try:
#             response  = requests.get(f"https://www.goodreads.com/book/show/{ID}"
#                                  ,proxies={'http': f'{cur_proxy}', 'https': f'{cur_proxy}'}
#                                  ,timeout=5)
#             print(ID)
#             break
#         except Exception as e:
# #             print(threadName, "Error: ", e)
#             continue
#     if response.status_code != 200:
#         try:
#             response  = requests.get(f"https://www.goodreads.com/book/show/{ID}")
#         except:
            
#     soup = bs4.BeautifulSoup(response.text, "html.parser")
#     genre_list = soup.find_all("a", {"class": "bookPageGenreLink"})
#     genre_amount = soup.find_all("div", {"class": "bookPageGenreLink"})
#     for i in range(min(len(genre_list),len(genre_amount))):
#         genre_string += genre_list[i].text
#         m = re.search(r"(\d{0,3},?\d{0,3},?\d{0,3}) users",genre_amount[i].text)
#         genre_users_string +=  m.group(1)

#         genre_string += "|"
#         genre_users_string += "|"
#     print(f"Book ID:{ID}  Genre:{genre_string}")
#     dic_genre[ID] = genre_string
#     dic_genre_number[ID] = genre_users_string
# #     Genre_df.loc[Genre_df.Id == ID,"Genre"] = genre_string
# #     Genre_df.loc[Genre_df.Id == ID,"Genre_user_amount"] = genre_users_string
# # append queue           
# urls = Genre_df.GDreads_url[:10]
# workQueue = queue.Queue(len(URLs))
# for url in urls:
#     workQueue.put(url)

# threads = []
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#     executor.map(crawl, urls)

# # for i in range(1,17):
# #     # create 10 threads
# #     thread = myThread("Thread-" + str(i), q=workQueue)
# #     # open thread
# #     thread.start()
# #     # append new thread
# #     threads.append(thread)

# # # wait until all threads finish
# # for thread in threads:
# #     thread.join()

# end = time.time()
# print("Total times：{} s".format(end - start))


In [107]:
Genre_df_LangNotNa = Genre_df[~Genre_df.Language.isna()]

,Id,Name,RatingDist1,pagesNumber,RatingDist4,RatingDistTotal,PublishMonth,PublishDay,Publisher_x,CountsOfReview,...,PagesNumber,Book-Title,Book-Author,Year-Of-Publication,Publisher_y,Image-URL-S,Image-URL-M,Image-URL-L,Genre,Genre_user_amount
0,2,Harry Potter and the Order of the Phoenix (Har...,1:12455,870,4:604283,total:2358637,1,9,Scholastic Inc.,29770,...,NaN,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2004,Scholastic Paperbacks,http://images.amazon.com/images/P/0439358078.0...,http://images.amazon.com/images/P/0439358078.0...,http://images.amazon.com/images/P/0439358078.0...,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...,"53,478|16,460|15,017|4,162|3,033|2,212|2,071|2..."
1,4,Harry Potter and the Chamber of Secrets (Harry...,1:11896,352,4:706082,total:2560657,1,11,Scholastic,244,...,NaN,Harry Potter and the Chamber of Secrets (Harry...,J. K. Rowling,2003,Arthur A. Levine Books,http://images.amazon.com/images/P/0439554896.0...,http://images.amazon.com/images/P/0439554896.0...,http://images.amazon.com/images/P/0439554896.0...,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...,"58,010|16,758|16,408|4,587|3,893|3,098|2,525|2..."
2,5,Harry Potter and the Prisoner of Azkaban (Harr...,1:10128,435,4:630534,total:2610317,1,5,Scholastic Inc.,37093,...,NaN,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,2004,Scholastic Paperbacks,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,,
3,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,1:3443,815,4:75683,total:274268,30,4,Del Rey Books,4119,...,NaN,The Ultimate Hitchhiker's Guide to the Galaxy,DOUGLAS ADAMS,2002,Del Rey,http://images.amazon.com/images/P/0345453743.0...,http://images.amazon.com/images/P/0345453743.0...,http://images.amazon.com/images/P/0345453743.0...,Science Fiction|Fiction|Humor|Fantasy|Classics...,"3,417|2,792|1,477|1,160|819|474|416|182|179|106|"
4,18,The Ultimate Hitchhiker's Guide (Hitchhiker's ...,1:3443,815,4:75683,total:274268,17,1,Wings Books,194,...,NaN,The Ultimate Hitchhiker's Guide,Douglas Adams,1996,Random House Value Publishing,http://images.amazon.com/images/P/0517149257.0...,http://images.amazon.com/images/P/0517149257.0...,http://images.amazon.com/images/P/0517149257.0...,Fail connection,Fail connection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141446,999298,The Genesis Code: A Novel of Suspense,1:64,467,4:1397,total:3581,28,2,Ballantine Books,25,...,NaN,The Genesis Code,John Case,1998,Ballantine Books,http://images.amazon.com/images/P/0345422317.0...,http://images.amazon.com/images/P/0345422317.0...,http://images.amazon.com/images/P/0345422317.0...,,
141461,999644,Life's Greatest Lessons: 20 Things That Matter,1:8,167,4:227,total:705,7,1,Fireside,81,...,NaN,Life's Greatest Lessons: 20 Things That Matter,Hal Urban,2002,Fireside,http://images.amazon.com/images/P/074323782X.0...,http://images.amazon.com/images/P/074323782X.0...,http://images.amazon.com/images/P/074323782X.0...,,
141464,999665,"Pay Dirt (Mrs. Murphy, #4)",1:10,251,4:1374,total:3367,1,11,Bantam Books,4,...,NaN,"Pay Dirt, Or, Adventures at Ash Lawn",Rita Mae Brown,1995,Bantam,http://images.amazon.com/images/P/0553096036.0...,http://images.amazon.com/images/P/0553096036.0...,http://images.amazon.com/images/P/0553096036.0...,,
141471,999893,"Bell, Cook, and Candle (Angie Amalfi, #9)",1:7,336,4:71,total:211,8,1,Avon,10,...,NaN,"Bell, Cook, and Candle : An Angie Amalfi Myste...",Joanne Pence,2002,Avon,http://images.amazon.com/images/P/0061030848.0...,http://images.amazon.com/images/P/0061030848.0...,http://images.amazon.com/images/P/0061030848.0...,,


In [256]:
# record the previous disconnected index
break_point = 29810
# counting for disconnection to prevent blocking by the target server
count = 0
# to detect if the connection is failed 
pre_Book_Gerne = ""
# total book amount
book_amount = 46333
# already crawled amount
finished_amount = 13602         

START = time.ctime()
print(f"{START}")
for i,k in enumerate(Genre_df_LangNotNa.loc[break_point:].index):
    cur_id = Genre_df_LangNotNa.loc[k,"Id"]
    print(f"current id {cur_id}")
    print(f"current k {k}")
    Book_Gerne, Book_Gerne_amount = Crawler(cur_id, proxy_ips)
    if Book_Gerne == "":
        Book_Gerne, Book_Gerne_amount = Crawler(cur_id, proxy_ips)
    if (pre_Book_Gerne == Book_Gerne ==  "Fail connection"):
        count += 1
        print(f"connection failed {count} times")
    if count == 4:
        time.sleep(200)
    if count == 5:
        break
    if Book_Gerne !=  "Fail connection":
        count = 0
    Genre_df.loc[k,"Genre"] = Book_Gerne
    Genre_df.loc[k,"Genre_user_amount"] = Book_Gerne_amount
    if ((i+break_point) % 1414 == 0) & (i+break_point != 0):
        period = (i+break_point)//1414
        Genre_df.iloc[1414*(period-1)+finished_amount:1414*(period)+finished_amount,:].to_csv(f"books_LangNotNa_V2_{i}.csv")
        print("-"*90)
        print("file saved")
    print(f"Book number:{i+finished_amount} out of {book_amount} ({(finished_amount+i)/(book_amount)*100:.3f}%)")
    pre_Book_Gerne = Book_Gerne

print(f"{START}")
print(time.ctime())
Genre_df.to_csv(f"Current allLangNotNa books.csv")
temp = Genre_df.loc[~Genre_df.loc[:,"Language"].isna(),:]
temp.to_csv("All books with lang_Genre_5.csv")
print("-"*90)
print("file saved")

Mon May  2 04:24:36 2022
current id 1218136
current k 29810
Book number:13602 out of 46333 (29.357%)
current id 1218313
current k 29823
connection failed 1 times
Book number:13603 out of 46333 (29.359%)
current id 1218366
current k 29826
connection failed 2 times
Book number:13604 out of 46333 (29.361%)
current id 1218370
current k 29827
connection failed 3 times
Book number:13605 out of 46333 (29.364%)
current id 1218380
current k 29828
connection failed 4 times
Book number:13606 out of 46333 (29.366%)
current id 1218394
current k 29829
Book number:13607 out of 46333 (29.368%)
current id 1218434
current k 29834
Book number:13608 out of 46333 (29.370%)
current id 1218475
current k 29836
Book number:13609 out of 46333 (29.372%)
current id 1218526
current k 29844
Book number:13610 out of 46333 (29.374%)
current id 1218578
current k 29849
Book number:13611 out of 46333 (29.376%)
current id 1218601
current k 29850
Book number:13612 out of 46333 (29.379%)
current id 1218859
current k 29867


Book number:13700 out of 46333 (29.569%)
current id 1226696
current k 30206
Book number:13701 out of 46333 (29.571%)
current id 1226787
current k 30211
Book number:13702 out of 46333 (29.573%)
current id 1227044
current k 30228
Book number:13703 out of 46333 (29.575%)
current id 1227143
current k 30234
Book number:13704 out of 46333 (29.577%)
current id 1227159
current k 30237
Book number:13705 out of 46333 (29.579%)
current id 1227182
current k 30241
connection failed 1 times
Book number:13706 out of 46333 (29.582%)
current id 1227220
current k 30243
Book number:13707 out of 46333 (29.584%)
current id 1227259
current k 30244
Book number:13708 out of 46333 (29.586%)
current id 1227318
current k 30246
Book number:13709 out of 46333 (29.588%)
current id 1227457
current k 30257
Book number:13710 out of 46333 (29.590%)
current id 1227600
current k 30261
Book number:13711 out of 46333 (29.592%)
current id 1227612
current k 30262
Book number:13712 out of 46333 (29.594%)
current id 1227689
cu

In [257]:
temp = Genre_df.loc[~Genre_df.loc[:,"Language"].isna(),:]

In [258]:
# 6118
temp.loc[temp.Genre!="",:].loc[temp.Genre!="Fail connection",:].shape

(7915, 30)

In [264]:
# temp.loc[temp.Genre!="",:].loc[temp.Genre!="Fail connection",:].to_csv("Clean book data.csv")

In [7]:
# os.chdir("D:\\user\\Documents\\WebMining")

In [263]:
# Genre_df.to_csv("All_Bookdata.csv")